# Spam Mail

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving spam.csv to spam.csv


In [3]:
spam = pd.read_csv(filename, sep = ',', encoding = 'latin1')
spam.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [4]:
df = spam[['v1', 'v2']]
df['label'] = df.v1.apply(lambda x: 1 if x == 'spam' else 0)
df.head(3)

,v1,v2,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
 2   label   5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [6]:
df.shape

(5572, 3)

## Preprocessing

In [7]:
df.isnull().sum()

v1       0
v2       0
label    0
dtype: int64

In [8]:
df.v2.nunique()

5169

In [9]:
df.drop_duplicates('v2', keep = 'first', inplace = True)

In [10]:
df['content'] = df.v2.str.replace('[^A-Za-z ]', '')
df.head(3)

,v1,v2,label,content
0,ham,"Go until jurong point, crazy.. Available only ...",0,Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,0,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,Free entry in a wkly comp to win FA Cup final...


In [11]:
df['content'] = df.content.apply(lambda x: x.lower())

In [12]:
df.v1.value_counts()

ham     4516
spam     653
Name: v1, dtype: int64

## DTM

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.content, df.label, test_size = 0.2,
    stratify = df.label, random_state = 2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4135,), (1034,), (4135,), (1034,))

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Count Vectorizer

In [15]:
cvect = CountVectorizer(stop_words = 'english', ngram_range = (1, 1))
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((4135, 7147), (1034, 7147))

In [16]:
lr1 = LogisticRegression()
lr1.fit(X_train_cv, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
pred1 = lr1.predict(X_test_cv)

In [18]:
score1 = accuracy_score(y_test, pred1)
score1

0.9777562862669246

## TF-IDF

In [19]:
tvect = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 1))
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((4135, 7147), (1034, 7147))

In [20]:
lr2 = LogisticRegression()
lr2.fit(X_train_tv, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
pred2 = lr2.predict(X_test_tv)

In [22]:
score2 = accuracy_score(y_test, pred2)
score2

0.9593810444874274